In [1]:
# Step 1: Install CD-HIT (if not installed)
!conda install -y -c bioconda cd-hit

Channels:
 - bioconda
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.



In [3]:
# Step 2: Run CD-HIT to cluster sequences at 97% similarity
!cd-hit -i translated_proteins.fasta -o clustered_proteins.fasta -c 0.97 -n 5

Program: CD-HIT, V4.8.1 (+OpenMP), Nov 12 2024, 10:35:24
Command: cd-hit -i translated_proteins.fasta -o
         clustered_proteins.fasta -c 0.97 -n 5

Started: Fri Feb  7 19:55:51 2025
                            Output                              
----------------------------------------------------------------
total seq: 1817
longest and shortest : 2358 and 11
Total letters: 168514
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 0M
Buffer          : 1 X 16M = 16M
Table           : 1 X 65M = 65M
Miscellaneous   : 0M
Total           : 82M

Table limit with the given memory limit:
Max number of representatives: 2550757
Max number of word counting entries: 89709506

comparing sequences from          0  to       1817
.
     1817  finished       1747  clusters

Approximated maximum memory consumption: 83M
writing new database
writing clustering information
program completed !

Total CPU time 0.07


In [7]:
# Step 3: Parse the CD-HIT cluster file
import pandas as pd
from Bio import SeqIO

# Read CD-HIT clustering output
cluster_file = "clustered_proteins.fasta.clstr"
cluster_map = {}

with open(cluster_file, "r") as clstr_file:
    cluster_id = None
    for line in clstr_file:
        if line.startswith(">Cluster"):
            cluster_id = line.strip()  # Extract cluster ID
        else:
            seq_id = line.split(">")[1].split("...")[0].strip()  # Extract sequence ID
            cluster_map[seq_id] = cluster_id

# Step 4: Read the clustered protein sequences
seq_records = {rec.id: rec for rec in SeqIO.parse("clustered_proteins.fasta", "fasta")}

# Step 5: Select the longest sequence in each cluster
representative_seqs = {}
missing_seqs = []

for seq_id, cluster_id in cluster_map.items():
    if seq_id in seq_records:
        seq_length = len(seq_records[seq_id].seq)
        # Keep the longest sequence in the cluster
        if cluster_id not in representative_seqs or seq_length > len(representative_seqs[cluster_id].seq):
            representative_seqs[cluster_id] = seq_records[seq_id]
    else:
        missing_seqs.append(seq_id)

# Step 6: Save the representative protein sequences
with open("representative_proteins.fasta", "w") as rep_file:
    for rep_seq in representative_seqs.values():
        SeqIO.write(rep_seq, rep_file, "fasta")

# Step 7: Log any missing sequences (for debugging)
if missing_seqs:
    with open("missing_sequences.log", "w") as log_file:
        log_file.write("\n".join(missing_seqs))
    print(f"⚠️ Warning: {len(missing_seqs)} sequences in .clstr were not found in clustered_proteins.fasta.")
    print("Check 'missing_sequences.log' for details.")

print("✅ Step 2 completed: Representative sequences saved in 'representative_proteins.fasta'.")


⚠️ Warning: 70 sequences in .clstr were not found in clustered_proteins.fasta.
Check 'missing_sequences.log' for details.
✅ Step 2 completed: Representative sequences saved in 'representative_proteins.fasta'.
